# Imports

In [1]:
# Python Scripts
%run -i "Scripts/scripts.py"

# Machine learning models
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

# Pipeline
from sklearn.pipeline import Pipeline

# Gridsearch
from sklearn.model_selection import GridSearchCV

Using TensorFlow backend.
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pandas/core/frame.py:4110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


## Random Forest

In [2]:
random_forest = RandomForestClassifier(random_state=19, oob_score=True)
svm = SVC(random_state=19)

In [3]:
# Random Forest grid search
param_forest = {"n_estimators":[300, 400], "max_depth":[5, 10, 15, 20]}
grid_forest = GridSearchCV(random_forest, param_grid=param_forest, cv=10)

In [4]:
grid_forest.fit(X_train_new, y_train_new)

GridSearchCV(cv=10, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=True, random_state=19,
                                  

In [5]:
grid_forest.best_params_

{'max_depth': 20, 'n_estimators': 400}

In [7]:
grid_forest.cv_results_

{'mean_fit_time': array([12.18688412, 16.05780041, 21.97823305, 29.70882988, 30.41397171,
        40.52461584, 35.56753399, 69.62446673]),
 'std_fit_time': array([ 0.22008636,  0.12045271,  0.20528849,  0.16260372,  0.09380517,
         0.20901353,  0.27180396, 67.23866003]),
 'mean_score_time': array([0.13015814, 0.17128446, 0.21044025, 0.27990613, 0.3221777 ,
        0.42577553, 0.45924673, 0.6023747 ]),
 'std_score_time': array([0.00418601, 0.00393056, 0.00838524, 0.01367975, 0.02285747,
        0.03533156, 0.02955256, 0.04479299]),
 'param_max_depth': masked_array(data=[5, 5, 10, 10, 15, 15, 20, 20],
              mask=[False, False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_n_estimators': masked_array(data=[300, 400, 300, 400, 300, 400, 300, 400],
              mask=[False, False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'max_depth': 5, 'n_estimators': 30

In [17]:
# Best mean test score
grid_forest.best_score_

0.7756272088196796

In [14]:
display(grid_forest.score(X_train_new, y_train_new))
display(grid_forest.score(X_test, y_test))

0.9308391307303587

0.700141898864809

### Confusion Matrix

In [18]:
# Able to determine metrics for a confusion matrix
def confusion_matrix_metrics(TN:int, FP:int, FN:int, TP:int, P:int, N:int):
    print("TNR:", TN/N)
    print("FPR:", FP/N)
    print("FNR:", FN/P)
    print("TPR:", TP/P)
    print("Precision:", TP/(TP+FP))  # Of the points your model predicted to be positive, how many of those points are truly positive
    print("Recall:", TP/(TP+FN))  # Of all the actually positive points, how many of them did the model predict as positive
    print("F1 Score:", (2*TP)/((2*TP) + (FP + FN)))  # the harmonic mean of precision and recall and is a better measure than accuracy

In [19]:
print('Confusion Matrix - Training Dataset')
print(pd.crosstab(y_train_new, grid_forest.predict(X_train_new), rownames=['True'], colnames=['Predicted'], margins=True))

Confusion Matrix - Training Dataset
Predicted      0      1    All
True                          
0          39091   5038  44129
1           1066  43063  44129
All        40157  48101  88258


In [20]:
confusion_matrix_metrics(TN=39091, FP=5038, FN=1066, TP=43063, P=44129, N=44129)

TNR: 0.8858347118674794
FPR: 0.11416528813252057
FNR: 0.02415645040676199
TPR: 0.975843549593238
Precision: 0.8952620527639757
Recall: 0.975843549593238
F1 Score: 0.9338176298384474


In [21]:
print('Confusion Matrix - Testing Dataset')
print(pd.crosstab(y_test, grid_forest.predict(X_test), rownames=['True'], colnames=['Predicted'], margins=True))

Confusion Matrix - Testing Dataset
Predicted     0      1    All
True                         
0           821   3688   4509
1           961  10034  10995
All        1782  13722  15504


In [22]:
confusion_matrix_metrics(TN=821, FP=3688, FN=961, TP=10034, P=10995, N=4509)

TNR: 0.18208028387669106
FPR: 0.8179197161233089
FNR: 0.08740336516598454
TPR: 0.9125966348340154
Precision: 0.7312345139192538
Recall: 0.9125966348340154
F1 Score: 0.8119108306024194


## Support Vector Machine

In [9]:
# Support Vector Machine grid search
param_svm = {"C":[0.1, 1.0, 100, 1000]}
grid_svm = GridSearchCV(svm, param_grid=param_svm)

In [ ]:
grid_svm.fit(X_train_rescaled, y_train)